### Import Package

In [29]:
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
# Category Encoder (categorical variable)
from sklearn.preprocessing import LabelEncoder
# Min max normalization (numerical variable)
from sklearn.preprocessing import MinMaxScaler
# Model (can also view the scikit learn to choose the model)
from xgboost import XGBRegressor
os.getcwd()

'G:\\我的雲端硬碟\\人工智慧\\workspace\\hw1'

### Read data

In [81]:
# df = pd.read_csv(r'./garments_worker_productivity.csv')
df = pd.read_csv(r'./train.csv')
print('viwe the data shape', df.shape)
df.head()

viwe the data shape (957, 15)


,date,quarter,department,day,team,targeted_productivity,smv,wip,over_time,incentive,idle_time,idle_men,no_of_style_change,no_of_workers,actual_productivity
0,1/1/2015,Quarter1,sweing,Thursday,8,0.80,26.16,1108.0,7080,98,0.0,0,0,59.0,0.940725
1,1/1/2015,Quarter1,finishing,Thursday,1,0.75,3.94,NaN,960,0,0.0,0,0,8.0,0.886500
2,1/1/2015,Quarter1,sweing,Thursday,11,0.80,11.41,968.0,3660,50,0.0,0,0,30.5,0.800570
3,1/1/2015,Quarter1,sweing,Thursday,12,0.80,11.41,968.0,3660,50,0.0,0,0,30.5,0.800570
4,1/1/2015,Quarter1,sweing,Thursday,6,0.80,25.90,1170.0,1920,50,0.0,0,0,56.0,0.800382


### Check Missing Value

In [82]:
df.isna().sum()

date                       0
quarter                    0
department                 0
day                        0
team                       0
targeted_productivity      0
smv                        0
wip                      396
over_time                  0
incentive                  0
idle_time                  0
idle_men                   0
no_of_style_change         0
no_of_workers              0
actual_productivity        0
dtype: int64

In [83]:
#Fillna by mean in feature wip
df['wip'] = df['wip'].fillna((df['wip'].mean()))

In [84]:
df.describe()

,team,targeted_productivity,smv,wip,over_time,incentive,idle_time,idle_men,no_of_style_change,no_of_workers,actual_productivity
count,957.000000,957.000000,957.000000,957.000000,957.000000,957.000000,957.000000,957.000000,957.000000,957.000000,957.000000
mean,6.444096,0.733041,15.258892,1249.989305,4751.755486,26.734587,0.894462,0.352142,0.134796,35.112853,0.743013
std,3.450526,0.098310,11.016415,1551.108780,3502.992893,31.645668,14.208009,3.185402,0.413693,22.201220,0.176066
min,1.000000,0.070000,2.900000,7.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.233705
25%,3.000000,0.700000,3.940000,984.000000,1440.000000,0.000000,0.000000,0.000000,0.000000,10.000000,0.653598
50%,7.000000,0.750000,15.260000,1249.989305,4080.000000,0.000000,0.000000,0.000000,0.000000,34.000000,0.799963
75%,9.000000,0.800000,22.940000,1249.989305,6960.000000,50.000000,0.000000,0.000000,0.000000,57.000000,0.850532
max,12.000000,0.800000,54.560000,23122.000000,25920.000000,138.000000,300.000000,45.000000,2.000000,89.000000,1.120437


### Categorical Variable

In [85]:
le = LabelEncoder()
df['date'] = le.fit_transform(df['date'])
df['quarter'] = le.fit_transform(df['quarter'])
df['department'] = le.fit_transform(df['department'])
df['day'] = le.fit_transform(df['day'])

### Numerical Variable

In [86]:
X = df[df.columns[0:-1]]
y = df[df.columns[-1]]

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
X_scaled

array([[0.        , 0.        , 1.        , ..., 0.        , 0.        ,
        0.65517241],
       [0.        , 0.        , 0.5       , ..., 0.        , 0.        ,
        0.06896552],
       [0.        , 0.        , 1.        , ..., 0.        , 0.        ,
        0.32758621],
       ...,
       [0.87234043, 0.75      , 1.        , ..., 0.        , 0.5       ,
        0.64367816],
       [0.87234043, 0.75      , 0.        , ..., 0.        , 0.        ,
        0.08045977],
       [0.87234043, 0.75      , 0.        , ..., 0.        , 0.        ,
        0.06896552]])

### Training (model construction)

### Xgboost

In [44]:
from sklearn.model_selection import RandomizedSearchCV

# XGB Regressor
# model = XGBRegressor() # you can also try hyperparameter
params = { 'subsample': np.arange(0.5, 1.0, 0.1),
           'n_estimators': [100, 250, 500, 1000],
           'max_depth': np.arange(10, 50, 5),
           'learning_rate': [0.01, 0.1, 0.01],
           'colsample_bytree': np.arange(0.5, 1.0, 0.1),
           'colsample_bylevel': np.arange(0.5, 1.0, 0.1)}
xgbr = XGBRegressor(seed = 42)
model = RandomizedSearchCV(estimator=xgbr,
                         param_distributions=params,
                         scoring='neg_mean_squared_error',
                         n_iter=10,
                         verbose=1)
model.fit(X_scaled,y)
print("Best parameters:", model.best_params_)
print("Lowest RMSE: ", (-model.best_score_)**(1/2.0))

Fitting 5 folds for each of 5 candidates, totalling 25 fits
Best parameters: {'subsample': 0.5, 'n_estimators': 500, 'max_depth': 30, 'learning_rate': 0.01, 'colsample_bytree': 0.7, 'colsample_bylevel': 0.7999999999999999}
Lowest RMSE:  0.13977348274183157


RandomizedSearchCV(estimator=XGBRegressor(base_score=None, booster=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None,
                                          enable_categorical=False, gamma=None,
                                          gpu_id=None, importance_type=None,
                                          interaction_constraints=None,
                                          learning_rate=None,
                                          max_delta_step=None, max_depth=None,
                                          min_child_weight=None, missing=nan,
                                          monotone_constraints=None,...
                                          validate_parameters=None,
                                          verbosity=None),
                   n_iter=5,
                   param_distributions={'colsample_byle

In [89]:
model = XGBRegressor(subsample=0.8,
                     n_estimators=1000,
                     max_depth=100,
                     learning_rate=0.01,
                     colsample_bytree=0.9,
                     colsample_bylevel=0.9)
model.fit(X_scaled,y)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=0.9,
             colsample_bynode=1, colsample_bytree=0.9, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.01, max_delta_step=0,
             max_depth=100, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=1000, n_jobs=16,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=0.8,
             tree_method='exact', validate_parameters=1, verbosity=None)

### Adaboost

In [68]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn import tree
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
# for depth in range (1,10):
#      tree_regressor=tree.DecisionTreeRegressor(max_depth=depth,random_state=1)
#      if tree_regressor.fit(X_scaled,y).tree_.max_depth<depth:
#          break
#      score=np.mean(cross_val_score(tree_regressor,X_scaled,y,scoring='neg_mean_squared_error', cv=5, n_jobs=1))
#      print(depth, score)

ada=AdaBoostRegressor()
search_grid={'n_estimators':[500,1000,2000],'learning_rate':np.arange(0.01, 0.1, 0.01),'random_state':np.arange(1, 10, 1)}
search=GridSearchCV(estimator=ada,param_grid=search_grid,scoring='neg_mean_squared_error',n_jobs=1,cv=2)
search.fit(X_scaled,y)
search.best_params_

{'learning_rate': 0.03, 'n_estimators': 500, 'random_state': 6}

In [69]:
model=AdaBoostRegressor(n_estimators=500,learning_rate=0.03,random_state=6)

### Cross validation
- usually we will do cross validation to evaluate the generalization of model
- or you can just use ```sklearn.model_selection.train_test_split``` to evaluate your model

In [90]:
from sklearn import pipeline
import joblib
from sklearn.model_selection import cross_val_score, cross_validate
# directly use cross_val_score
scores = cross_val_score(model, X_scaled, y, cv=5)
print("R2: %0.2f (std: %0.2f)" % (scores.mean(), scores.std()))
print(scores)
# need other scoring
scores = cross_validate(model, X, y, cv=5, scoring=('r2', 'neg_mean_squared_error', 'neg_mean_absolute_percentage_error'))
print("====================================")
print("R2: {} (std: {})".format(scores['test_r2'].mean(), scores['test_r2'].std()))
print("MSE: {} (std: {})".format(-scores['test_neg_mean_squared_error'].mean(), scores['test_neg_mean_squared_error'].std()))
print("MAPE: {} (std: {})".format(-scores['test_neg_mean_absolute_percentage_error'].mean(), scores['test_neg_mean_absolute_percentage_error'].std()))

R2: 0.32 (std: 0.11)
[0.27519377 0.22390267 0.49537609 0.3766233  0.21754863]


C:\Users\Wynns\anaconda3\envs\workspace\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\Wynns\anaconda3\envs\workspace\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\Wynns\anaconda3\envs\workspace\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\Wynns\anaconda3\envs\workspace\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Ind

R2: 0.3194280060341489 (std: 0.10701072498189004)
MSE: 0.02004866668312732 (std: 0.004695934788050171)
MAPE: 0.15489215218724492 (std: 0.038613440152886584)


### Prediction
1. read data
2. model prediction
3. to_csv

In [91]:
df_test = pd.read_csv(r'./test.csv')
print(df_test.shape)
df_test.head()

(240, 14)


,date,quarter,department,day,team,targeted_productivity,smv,wip,over_time,incentive,idle_time,idle_men,no_of_style_change,no_of_workers
0,2/26/2015,Quarter4,sweing,Thursday,4,0.80,30.10,437.0,7080,32,0.0,0,2,59
1,2/26/2015,Quarter4,sweing,Thursday,5,0.35,27.48,413.0,6840,38,0.0,0,1,57
2,2/26/2015,Quarter4,finishing,Thursday,10,0.70,2.90,NaN,3360,0,0.0,0,0,8
3,2/26/2015,Quarter4,finishing,Thursday,9,0.75,2.90,NaN,960,0,0.0,0,0,8
4,2/26/2015,Quarter4,sweing,Thursday,1,0.35,26.66,1164.0,6600,23,0.0,0,2,55


### Same preprocessing step as training procedure
- can write it like a function or pipeline

In [92]:
# Category Encoder
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df_test['date'] = le.fit_transform(df_test['date'])
df_test['quarter'] = le.fit_transform(df_test['quarter'])
df_test['department'] = le.fit_transform(df_test['department'])
df_test['day'] = le.fit_transform(df_test['day'])

In [93]:
X_test = df_test[df_test.columns]
# Numerical Variable
scaler = MinMaxScaler()
X_test_scaled = scaler.fit_transform(X_test)
y_pred = model.predict(X_test_scaled)

In [94]:
model

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=0.9,
             colsample_bynode=1, colsample_bytree=0.9, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.01, max_delta_step=0,
             max_depth=100, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=1000, n_jobs=16,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=0.8,
             tree_method='exact', validate_parameters=1, verbosity=None)

### Output .csv for submission

In [95]:
df_ans = pd.DataFrame(y_pred.astype(float), columns=['actual_productivity'])
df_ans.to_csv('mySubmission.csv', index_label='Id')

### Some Tips
1. try different scaling approach (numerical variable)
2. encoding method
3. different model
4. feature engineering (some different transformation)
5. do some data exploration to see other pattern
6. hyperparameter

### Other Complementary
- pipeline: https://scikit-learn.org/stable/modules/compose.html#pipeline
- also can try some basic deep learning model (ANN)